In [10]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.chat_models.ollama import ChatOllama
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain.storage.file_system import LocalFileStore
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain_core.runnables.base import RunnableLambda
from langchain.memory.buffer import ConversationBufferMemory
from langchain_core.output_parsers.string import StrOutputParser


LLM_model = "ollama"
# LLM_model = "openai"

file_name = "document.txt"

if LLM_model == "openai":
    # 챗 지피티
    models = "gpt-3.5-turbo"
    llm = ChatOpenAI(
        temperature=0.1,
        model=models,
        streaming=True,
        callbacks=[
            StreamingStdOutCallbackHandler(),
        ],
    )
elif LLM_model == "ollama":
    # 로컬 LLM (ollama)
    models = "openhermes:latest"
    llm = ChatOllama(
        temperature=0.1,
        model=models,
        streaming=True,
        callbacks=[
            StreamingStdOutCallbackHandler(),
        ],
    )

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)


loader = UnstructuredFileLoader(f"./files/{file_name}")

cache_dir = LocalFileStore(f"./.cache/embeddings/{LLM_model}/{models}/{file_name}")

docs = loader.load_and_split(text_splitter=splitter)

if LLM_model == "openai":
    embeddings = OpenAIEmbeddings()

elif LLM_model == "ollama":
    embeddings = OllamaEmbeddings(
        model=models,
    )

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


ouput_parser = StrOutputParser()

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory),
    }
    | prompt
    | llm
    | ouput_parser
)


def invoke_chain(question):
    result = chain.invoke({"question": question}).content
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(type(result))


invoke_chain("Is Aaronson guilty?")

Yes, Aaronosn is guilty. The text provides evidence that he committed the crime of thoughtcrime by having unauthorized thoughts about Julia and Winston. This is a serious offense in the totalitarian society they live in, as evidenced by his arrest and interrogation by O'Brien. Additionally, Aaronson's confession to O'Brien further confirms his guilt.<class 'str'>


In [3]:
invoke_chain("What message did he write in the table?")

The text provided does not contain any information about a message written on a table. The passage discusses O'Brien and Winston's conversation about power, control, and the nature of reality.

In [4]:
invoke_chain("Who is Julia?")

Julia is a character in George Orwell's novel "1984." She is Winston Smith's lover and fellow rebel against the Party. Julia is described as being attractive, with dark hair and eyes, and she works at the Fiction Department of the Ministry of Truth. Throughout the novel, Julia and Winston engage in a secret affair, sharing their rebellious thoughts and dreams of overthrowing the Party. However, their relationship is ultimately used by the Party to break down their individual resistance and force them into conformity.

In [5]:
invoke_chain("What was the first question that i ask you?")

The first question you asked O'Brien in this passage is, "You have read it?", referring to The Book (Goldstein's book).